In [2]:
from dotenv import load_dotenv


load_dotenv("openai.env")

True

- ZERO_SHOT_REACT_DESCRIPTION
- CHAT_ZERO_SHOT_REACT_DESCRIPTION
- CONVERSATIONAL_REACT_DESCRIPTION
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION
- OPENAI_FUNCTIONS
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION

# ZERO_SHOT_REACT_DESCRIPTION
零样本增强式生成，即在没有示例的情况下可以自主的进行对话的类型。https://blog.csdn.net/zcyzcyjava/article/details/127006287 [零样本、单样本、少样本]

In [3]:
from langchain.llms import OpenAI
from langchain.agents import (load_tools, initialize_agent, AgentType)
import os

# serpapi申请的密钥
os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"

llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
# print(agent)
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")



> Entering new AgentExecutor chain...


APIConnectionError: Connection error.

# CHAT_ZERO_SHOT_REACT_DESCRIPTION 
- 使用了chatmodel

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)



verbose=True tags=['chat-zero-shot-react-description'] agent=ChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Search, Calculator\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_N

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")



> Entering new AgentExecutor chain...
Question: Who is the current President of the United States? What is his age divided by 2?

Thought: I should search for the current President of the United States and then calculate his age divided by 2.

Action:
```
{
  "action": "Search",
  "action_input": "current President of the United States"
}
```
Observation: Joe Biden
Thought:Now I should search for Joe Biden's age and divide it by 2.

Action:
```
{
  "action": "Search",
  "action_input": "Joe Biden age"
}
```

Observation: 81 years
Thought:```
{
  "action": "Calculator",
  "action_input": "81 / 2"
}
```
Observation: Answer: 40.5
Thought:Final Answer: Joe Biden is the current President of the United States. His age divided by 2 is 40.5.

> Finished chain.


{'input': '现在美国总统是谁？他的年龄除以2是多少？',
 'output': 'Joe Biden is the current President of the United States. His age divided by 2 is 40.5.'}

# CONVERSATIONAL_REACT_DESCRIPTION 
- 一个对话型的agent，这个agent要求与memory一起使用

In [16]:
from langchain.llms import OpenAI
from langchain.agents import (load_tools, initialize_agent, AgentType)
from langchain.memory import ConversationBufferMemory
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,  #记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)

memory=ConversationBufferMemory(memory_key='chat_history') verbose=True tags=['conversational-react-description'] agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Ass

In [17]:
agent.invoke("hi i am tomie")



> Entering new AgentExecutor chain...

AI: Hello Tomie, it's nice to meet you! How can I assist you today?

> Finished chain.


{'input': 'hi i am tomie',
 'chat_history': '',
 'output': "Hello Tomie, it's nice to meet you! How can I assist you today?"}

In [18]:
agent.invoke("what is my name?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "what is my name"
Observation: ['My Name Is type: Song by Eminem.', 'My Name Is entity_type: music, music.', 'My Name Is kgmid: /g/1s05n4sxn.', 'My Name Is album: The Slim Shady LP.', 'My Name Is released: 1999.', 'My Name Is artist: Eminem.', 'My Name Is genres: Spoken Word, Hip-Hop/Rap.', 'My Name Is awards: Grammy Award for Best Rap Solo Performance, MTV Video Music Award for Best New Artist.', 'Disney+ is the only place to stream your favorites from Disney, Pixar, Marvel, Star Wars, National Geographic and more.', "REMASTERED IN HD! Get Rihanna's eighth studio album ANTI now: Download on TIDAL: http://smarturl.it/downloadANTI Stream on TIDAL: ...", '"What\'s my name? What\'s my name? Say it louder! Sing along to ""What\'s My Name"" with China Anne McClain from Descendants 2 and watch ...', 'Enter the username(s) in the search box, select any category filters & click the search

{'input': 'what is my name?',
 'chat_history': "Human: hi i am tomie\nAI: Hello Tomie, it's nice to meet you! How can I assist you today?",
 'output': 'Your name is Tomie. Is there anything else you would like to know?'}

In [19]:
agent.invoke("中国陕西西安现在的气温多少？截止目前我们聊了什么？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: What is the current temperature in Xi'an, Shaanxi, China?
Observation: {'type': 'weather_result', 'temperature': '72', 'unit': 'Fahrenheit', 'precipitation': '1%', 'humidity': '54%', 'wind': '4 mph', 'location': "Xi'An, Shaanxi, China", 'date': 'Thursday 9:00 PM', 'weather': 'Clear with periodic clouds'}
Thought: Do I need to use a tool? No
AI: We have discussed your name and the current temperature in Xi'an, Shaanxi, China. Is there anything else you would like to talk about?

> Finished chain.


{'input': '中国陕西西安现在的气温多少？截止目前我们聊了什么？',
 'chat_history': "Human: hi i am tomie\nAI: Hello Tomie, it's nice to meet you! How can I assist you today?\nHuman: what is my name?\nAI: Your name is Tomie. Is there anything else you would like to know?",
 'output': "We have discussed your name and the current temperature in Xi'an, Shaanxi, China. Is there anything else you would like to talk about?"}

# CHAT_CONVERSATIONAL_REACT_DESCRIPTION 
- 使用了chatmodel

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (load_tools, initialize_agent, AgentType)
from langchain.memory import ConversationBufferMemory
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,  #记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)
print(agent.agent.llm_chain.prompt.messages[3])

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['chat-conversational-react-description'] agent=ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Ass

In [21]:
agent.invoke("hi i am tony")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Tony! How can I assist you today?"
}
```

> Finished chain.


{'input': 'hi i am tony',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony! How can I assist you today?')],
 'output': 'Hello Tony! How can I assist you today?'}

In [22]:
agent.invoke(input="what is my name?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Your name is Tony."
}
```

> Finished chain.


{'input': 'what is my name?',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony! How can I assist you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Tony.')],
 'output': 'Your name is Tony.'}

In [23]:
agent.invoke(input="我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "1998年世界杯冠军"
}
```
Observation: France national football team
Thought:```json
{
    "action": "Final Answer",
    "action_input": "你名字的最后一个字母是 Y。1998年的世界杯冠军是法国国家足球队。"
}
```

> Finished chain.


{'input': '我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony! How can I assist you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Tony.'),
  HumanMessage(content='我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？'),
  AIMessage(content='你名字的最后一个字母是 Y。1998年的世界杯冠军是法国国家足球队。')],
 'output': '你名字的最后一个字母是 Y。1998年的世界杯冠军是法国国家足球队。'}

# OPENAI_FUNCTIONS
- 使用openai的函数调用来实现的，只支持openai模型。

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,  #记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.prompt.messages)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['openai-functions'] agent=OpenAIFunctionsAgent(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12543edd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1253f5710>, model_name='gpt-4-1106-preview', temperature=0.0, openai_api_key='sk-bQGeRZevfQI95TPb1b1a3e80011c4f368150270f687dF163', openai_api_base='https://api.aihubmix.com/v1', openai_proxy='https://api.aihubmix.com/v1'), tools=[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719', aiosession=None)>

In [25]:
agent.invoke(input="今天比特币的价格是多少？它的价格开平方是多少？")



> Entering new AgentExecutor chain...

Invoking: `Search` with `
Bitcoin price today`


63,283.40
Invoking: `Calculator` with `sqrt(63283.40)`


Answer: 251.56192080678665今天比特币的价格是 $63,283.40。它的价格开平方是大约 $251.56。

> Finished chain.


{'input': '今天比特币的价格是多少？它的价格开平方是多少？',
 'chat_history': [HumanMessage(content='今天比特币的价格是多少？它的价格开平方是多少？'),
  AIMessage(content='今天比特币的价格是 $63,283.40。它的价格开平方是大约 $251.56。')],
 'output': '今天比特币的价格是 $63,283.40。它的价格开平方是大约 $251.56。'}

# STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 
- 对输出做了结构化处理

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (load_tools, initialize_agent, AgentType)
from langchain.memory import ConversationBufferMemory
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,  #记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[1].prompt.template)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['structured-chat-zero-shot-react-description'] agent=StructuredChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {{\'tool_input\': {{\'type\': \'string\'}}}}\nCalculator: Useful for when you need to answer questions about math., args: {{\'tool_input\': {{\'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Search, Calculator\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n

In [31]:
agent.invoke({"input":"what is langchain?"})



> Entering new AgentExecutor chain...
Thought: Langchain is not a widely recognized term that I am familiar with, and it may refer to a specific product, service, or concept that has emerged after my last update. I will use the search tool to find the most current information on Langchain.

Action:
```
{
  "action": "Search",
  "action_input": {"type": "string"}
}
```
Observation: ['Search Tool for the Retrieval of Interacting Genes/Proteins', 'STRING entity_type: kp3_verticals.', 'STRING kgmid: /m/05c539z.', 'STRING download_url: url.', 'STRING primary_citation: PMID 30476243.', 'STRING research_center: Academic Consortium.', 'STRING version: 11.5 (August 2021).', 'STRING web_service_url: rest.', 'STRING is a Core Data Resource as designated by Global Biodata Coalition and ELIXIR.', 'verb · 1. a. : to equip with strings. b. : to tune the strings of · 2. : to make tense : key up · 3. a. : to thread on or as if on a string. b.', 'Encouraging creativity since 1949. We create and produc

ToolException: Too many arguments to single-input tool Search.
                Consider using StructuredTool instead. Args: ['string', 'what is Langchain']